In [1]:
#import neccessary modules
# uuid is used to generate unique identifiers
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import uuid

In [2]:
# helper functions
from timeseries_helpers import datasetstorer
from timeseries_helpers import dataset_importer
from timeseries_helpers import database_importer
from timeseries_helpers import standardizer
from data_preprocessing import extract_accelerometer_data, csv_to_dataset_list
from timeseries_helpers import classificationHelpers
import data

In [3]:
# Read the CSV files

data_2 = pd.read_csv(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\data\Max_Kinnhacken.csv')
data_1 = pd.read_csv(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\data\Max_Gerade.csv')
data_3 = pd.read_csv(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\data\Max_Kopfhacken.csv')
data_4 = pd.read_csv(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\data\Max_Gemischt_01.csv')
data_5 = pd.read_csv(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\data\Max_Gemischt_02.csv')

# List of CSV file paths
csv_files = glob.glob(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\data\*.csv')

# List to store each CSV file as a DataFrame
dfs = []

# Loop through the list of CSV file paths
for csv_file in csv_files:
    # Read CSV file into DataFrame and append to list
    dfs.append(pd.read_csv(csv_file))

# Concatenate all DataFrames in the list into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)

# Write the merged dataframe to a new CSV file
merged_df.to_csv(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\jupyter_tutorials-master\Merged_df.csv', index=False)

ds = csv_to_dataset_list(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\jupyter_tutorials-master\Merged_df.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 216456 entries, 0 to 216461
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   timestamp  216456 non-null  float64
 1   x          216456 non-null  float64
 2   y          216456 non-null  float64
 3   z          216456 non-null  float64
dtypes: float64(4)
memory usage: 8.3 MB
Data imported without any errors!
 ##### Information: ####
 Imported datasets: 1


C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\venv\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [4]:
import os

directory = r'\Users\raouldoublan\Documents\GitHub\Boxschlag-Klassifikation'

# Create the directory if it does not exist
if not os.path.exists(directory):
    os.makedirs(directory)

# Now you can save your file in the 'directory'

In [5]:
data = extract_accelerometer_data(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\data\Max_Gerade.csv')

def csv_to_json(file_path, ds = []):
    # Read the CSV file and morph it into a indexed Dictionary
    data = extract_accelerometer_data(file_path)
    
    # create raws
    raws = []
    for idx, row in data.iterrows():
        raw = {'_id' : idx, 'timestamp' : row['timestamp'], 'x' : row['x'], 'y' : row['y'], 'z' : row['z']}
        raws.append(raw)

        
    # insert raws into dataset
    dataset = {'raws': raws, 'label': 0, 'count': len(data)}
    
    # append dataset to dataset list
    ds.append(dataset)
    
    return ds
    
    

In [6]:
data = extract_accelerometer_data(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\data\Max_Gerade.csv')

print(data.iloc[1]['x'])

-0.021240234375


In [7]:
data = csv_to_json(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\data\Max_Gerade.csv')
ds = extract_accelerometer_data(r'C:\Users\Raoul\Documents\GitHub\Boxschlag-Klassifikation\data\Max_Gerade.csv')

#print(ds)
print(data[0])

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
## period length in ms
periodLengthMS = 1000

## sample rate in µs
sampleRateUS = 10000

## test/train ratio
trainDataRatio = 0.7
trainDataAbs = 100*trainDataRatio

## Json dataset path
jsonDSpath = '/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Merged_Data.json'

In [9]:
#with open('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Merged_Data.json', 'r') as f:
#    jsnDataset = json.load(f)

tweets = []
with open('/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Merged_Data.json', 'r') as file:
    for line in file:
        tweets.append(json.loads(line))

dl = database_importer.jsonData_to_dataset_in_timedifference_us(tweets)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/raouldoublan/Documents/GitHub/Boxschlag-Klassifikation/data/Merged_Data.json'

In [ ]:
buff = classificationHelpers.prepareDataset(jsonDSpath, periodLengthMS, sampleRateUS)

In [ ]:
# converts the given list of dataframes to one dataframe
df_list = []
for idx,e in enumerate(ds):
    df = e.assign(punch_id=idx)
    df['label'] = 0
    df_list.append(df)
df_res = pd.concat(df_list)
df_new = df_res.reset_index(drop=True)
df_new = df_new.rename(index=str, columns={"accelerometerAccelerationX(G)": "a_x", "accelerometerAccelerationY(G)": "a_y", "accelerometerAccelerationZ(G)":"a_z"})

In [ ]:
df_new.head()